In [40]:
import requests
import xlrd
import json
import pandas as pd
from sqlalchemy import create_engine
from config import * 

In [41]:
url = "https://data.gov.au/data/api/3/action/datastore_search?resource_id=c6b211ad-3aa2-4f53-8427-01b52a6433a7"
response = requests.get(url).json()
print(json.dumps(response, indent=2))

{
  "help": "https://data.gov.au/data/api/3/action/help_show?name=datastore_search",
  "success": true,
  "result": {
    "include_total": true,
    "resource_id": "c6b211ad-3aa2-4f53-8427-01b52a6433a7",
    "fields": [
      {
        "type": "int",
        "id": "_id"
      },
      {
        "type": "text",
        "id": "Location_Name"
      },
      {
        "type": "text",
        "id": "Identifier"
      },
      {
        "type": "text",
        "id": "Road_Section"
      },
      {
        "type": "text",
        "id": "Public_Interest_Premises"
      },
      {
        "type": "text",
        "id": "State"
      },
      {
        "type": "numeric",
        "id": "Latitude"
      },
      {
        "type": "numeric",
        "id": "Longitude"
      },
      {
        "type": "text",
        "id": "Electorate"
      },
      {
        "type": "text",
        "id": "Local_Government_Area"
      },
      {
        "type": "numeric",
        "id": "Post_Code"
      },
      {
  

In [42]:
location = []
identifier = []
state = []
lga = []
electorate = []
latitude = []
longitude = []

url_query = "https://data.gov.au/data/api/3/action/datastore_search?resource_id=c6b211ad-3aa2-4f53-8427-01b52a6433a7&offset=0&limit=14000"

response = requests.get(url_query).json()    
length = len(response["result"]["records"])
    
for x in range(length):

    location.append(response["result"]["records"][x]["Location_Name"])
    identifier.append(response["result"]["records"][x]["Identifier"])
    state.append(response["result"]["records"][x]["State"])
    latitude.append(response["result"]["records"][x]["Latitude"])
    longitude.append(response["result"]["records"][x]["Longitude"]) 
    lga.append(response["result"]["records"][x]["Local_Government_Area"])
    electorate.append(response["result"]["records"][x]["Electorate"])

In [43]:
length

13480

In [44]:
blackspot_df = pd.DataFrame({"location": location, "identifier": identifier, "state": state, "local_government_area": lga, "electorate":
                            electorate, "lat": latitude, "lon": longitude})
blackspot_df

,location,identifier,state,local_government_area,electorate,lat,lon
0,Cotter River Recreation Area,ACT-0001,Australian Capital Territory,Unincorporated ACT,Bean,-35.324887,148.939594
1,Cotter Road,ACT-0002,Australian Capital Territory,Unincorporated ACT,Bean,-35.338535,148.990594
2,National Equestrian Centre,ACT-0003,Australian Capital Territory,Unincorporated ACT,Bean,-35.356065,149.014995
3,Stromlo,ACT-0004,Australian Capital Territory,Unincorporated ACT,Bean,-35.326666,149.000622
4,Uriarra Village,ACT-0005,Australian Capital Territory,Unincorporated ACT,Bean,-35.296238,148.924429
...,...,...,...,...,...,...,...
13475,Chidlow,WA-1762,Western Australia,Mundaring,Hasluck,-31.840440,116.241860
13476,Gidgegannup,WA-1763,Western Australia,Swan,Hasluck,-31.772370,116.232400
13477,Digger's Rest Station,WA-1764,Western Australia,Wyndham-East Kimberley,Durack,-15.638640,128.080260
13478,Gibb River Road,WA-1765,Western Australia,Wyndham-East Kimberley,Durack,-15.729220,127.821690


In [18]:
# act_index = blackspot_df.loc[blackspot_df["local_government_area"] == "Unincorporated ACT"].index
# act_index

Int64Index([0, 1, 2, 3, 4, 6, 7, 861, 2825], dtype='int64')

In [19]:
# blackspot_df = blackspot_df.drop(act_index)
# blackspot_df

,location,identifier,state,local_government_area,electorate,lat,lon
5,Booderee National Park,ACT-0006,Australian Capital Territory,Unincorp. Other Territories,Fenner,-35.133781,150.686178
8,Christmas Island,ET-0001,External Territory,Unincorp. Other Territories,Lingiari,-10.490846,105.622456
9,Cocos (Keeling) Islands,ET-0002,External Territory,Unincorp. Other Territories,Lingiari,-12.185414,96.827896
10,Norfolk Island,ET-0003,External Territory,Unincorp. Other Territories,Bean,-29.040835,167.954661
11,Aberbaldie,NSW-0001,New South Wales,Walcha,New England,-31.056603,151.503098
...,...,...,...,...,...,...,...
13475,Chidlow,WA-1762,Western Australia,Mundaring,Hasluck,-31.840440,116.241860
13476,Gidgegannup,WA-1763,Western Australia,Swan,Hasluck,-31.772370,116.232400
13477,Digger's Rest Station,WA-1764,Western Australia,Wyndham-East Kimberley,Durack,-15.638640,128.080260
13478,Gibb River Road,WA-1765,Western Australia,Wyndham-East Kimberley,Durack,-15.729220,127.821690


In [46]:
no_match_list = ["Unincorporated ACT", "Unincorp. Other Territories", "Western Plains Regional", "Gundagai", "Mallala", 
        "Orroroo/Carrieton", "Break O'Day", "Glamorgan/Spring Bay", "Waratah/Wynyard", "Kalgoorlie/Boulder"]

blackspot_df = blackspot_df[~blackspot_df["local_government_area"].isin(no_match_list)]
blackspot_df


,location,identifier,state,local_government_area,electorate,lat,lon
11,Aberbaldie,NSW-0001,New South Wales,Walcha,New England,-31.056603,151.503098
12,Abercrombie National Park,NSW-0002,New South Wales,Oberon,Calare,-34.062876,149.737975
13,Abercrombie Road,NSW-0003,New South Wales,Oberon,Calare,-34.007954,149.790508
14,Abercrombie Road,NSW-0004,New South Wales,Oberon,Calare,-34.033964,149.825793
15,Abercrombie Road,NSW-0005,New South Wales,Oberon,Calare,-34.127104,149.779201
...,...,...,...,...,...,...,...
13475,Chidlow,WA-1762,Western Australia,Mundaring,Hasluck,-31.840440,116.241860
13476,Gidgegannup,WA-1763,Western Australia,Swan,Hasluck,-31.772370,116.232400
13477,Digger's Rest Station,WA-1764,Western Australia,Wyndham-East Kimberley,Durack,-15.638640,128.080260
13478,Gibb River Road,WA-1765,Western Australia,Wyndham-East Kimberley,Durack,-15.729220,127.821690


In [47]:
blackspot_lga_columns = ["location", "lat", "lon", "local_government_area"]
blackspot_transformed = blackspot_df[blackspot_lga_columns].copy()
blackspot_transformed.index.name = "bs_id"
blackspot_transformed

,location,lat,lon,local_government_area
bs_id,,,,
11,Aberbaldie,-31.056603,151.503098,Walcha
12,Abercrombie National Park,-34.062876,149.737975,Oberon
13,Abercrombie Road,-34.007954,149.790508,Oberon
14,Abercrombie Road,-34.033964,149.825793,Oberon
15,Abercrombie Road,-34.127104,149.779201,Oberon
...,...,...,...,...
13475,Chidlow,-31.840440,116.241860,Mundaring
13476,Gidgegannup,-31.772370,116.232400,Swan
13477,Digger's Rest Station,-15.638640,128.080260,Wyndham-East Kimberley


In [48]:
connection_string = "{}:{}@localhost:5432/LGA_db".format(username, password)
engine = create_engine(f'postgresql://{connection_string}')

In [49]:
engine.table_names()

['blackout_spots',
 'location_id',
 'local_gov_area',
 'jurisdiction',
 'population_numbers',
 'population_growth',
 'pop_growth_type',
 'population_density']

In [50]:
blackspot_transformed.to_sql(name='blackout_spots', con=engine, if_exists='append', index=True)

In [51]:
jurisdiction_cols =["local_government_area", "electorate", "state"]
jurisdiction_transformed = blackspot_df[jurisdiction_cols ].copy()
jurisdiction_transformed.index.name = "index"
jurisdiction_transformed

,local_government_area,electorate,state
index,,,
11,Walcha,New England,New South Wales
12,Oberon,Calare,New South Wales
13,Oberon,Calare,New South Wales
14,Oberon,Calare,New South Wales
15,Oberon,Calare,New South Wales
...,...,...,...
13475,Mundaring,Hasluck,Western Australia
13476,Swan,Hasluck,Western Australia
13477,Wyndham-East Kimberley,Durack,Western Australia


In [52]:
jurisdiction_transformed.to_sql(name='jurisdiction', con=engine, if_exists='append', index=True)